In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Maximazing the Dsiplay
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

water_treatment= pd.read_excel('/kaggle/input/water_treatment.xlsx',header=None)
print (water_treatment.head())

# PREPROCESSING

# Droping unneccessary columns
water_treatment= water_treatment.drop([0],axis=1)
print ('Number of columns and rows',water_treatment.shape)
print ('Type of each column',water_treatment.dtypes)


# Missing Values

# replacing '?' by 0
water_treatment=water_treatment.replace('?', 0)
# Converting all in to float
water_treatment = water_treatment.apply(lambda x: x.astype(np.float64), axis=1)
# Now we can have all the relevant statistics of each column
print (water_treatment.describe())

# Replacing '0' of each column with average value
water_treatment=water_treatment.replace(0.0,water_treatment.mean())
print (water_treatment.head())

from sklearn.preprocessing import StandardScaler
x=water_treatment.ix[:,0:38].values

x = StandardScaler().fit_transform(x)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

print (principalDf.head())

normalizing_data = principalDf.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
normalized_data = min_max_scaler.fit_transform(normalizing_data)
principalDf_normalized = pd.DataFrame(normalized_data)

# Taking random value of k to start
clusterNum = 3
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(principalDf_normalized)
labels = k_means.labels_
print(labels)

principalDf["Cluster"] = labels
print (principalDf.head())

X = principalDf.ix[:,0:2].values
y = principalDf['Cluster'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)
from sklearn.neighbors import KNeighborsClassifier
k = 4
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

yhat = neigh.predict(X_test)
print (yhat[0:5])

from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))

# FInding the best k
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

# Visualization 
import matplotlib.pyplot as plt
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()

# Best K
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

# Now for k=1, PCA
clusterNum = 1
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(principalDf_normalized)
labels = k_means.labels_
print(labels)

principalDf["Cluster"] = labels
print (principalDf.head())

# output file
output= principalDf[['Cluster']]
print (output.head())

# Saving in text format
output.to_csv(r'kmean_PCA.txt', sep='\t')







